In [1]:
# Pandas program to condense user ratings into averages in order to build user profiles
import glob
import pandas as pd
print("Pandas Version Number: " + pd.__version__)

Pandas Version Number: 1.4.2


In [2]:
# Find total number of genres
anime_list_df = pd.read_csv('../../../data/cleaned/anime_list.csv')
genres = []
for i, row in anime_list_df.iterrows():
    current_row_genres = row['Genres']
    current_row_genres_list = current_row_genres.split(", ")

    # Ensure there are no duplicate values by using the properties of a set
    genres = list(set(genres + current_row_genres_list))

# Sort and display genres along with total number of genres
genres.sort()
print("Number of genres: " + str(len(genres)))
display(genres)

Number of genres: 42


['Action',
 'Adventure',
 'Cars',
 'Comedy',
 'Dementia',
 'Demons',
 'Drama',
 'Ecchi',
 'Fantasy',
 'Game',
 'Harem',
 'Historical',
 'Horror',
 'Josei',
 'Kids',
 'Magic',
 'Martial Arts',
 'Mecha',
 'Military',
 'Music',
 'Mystery',
 'Parody',
 'Police',
 'Psychological',
 'Romance',
 'Samurai',
 'School',
 'Sci-Fi',
 'Seinen',
 'Shoujo',
 'Shoujo Ai',
 'Shounen',
 'Shounen Ai',
 'Slice of Life',
 'Space',
 'Sports',
 'Super Power',
 'Supernatural',
 'Thriller',
 'Unknown',
 'Vampire',
 'Yaoi']

In [3]:
# Create path to cleaned user_ratings
path = '../../../data/cleaned/user_ratings/'
user_ratings_files = glob.glob(path + "*.csv")

# Create new DataFrame to hold a user's average ratings
average_ratings_df = pd.DataFrame(columns=genres)
average_ratings_df.insert(0, "user_id", [])
display(average_ratings_df)

,user_id,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi


In [4]:
# Create DataFrame of all files
user_ratings_files_df = pd.read_csv('../../../data/test.csv') ##### Take this out and change to next commented line for real data input instead of test data
#user_ratings_files_df = pd.concat(map(pd.read_csv, user_ratings_files))
display(user_ratings_files_df)

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,1,22535,9
4,1,32281,10
5,1,38000,9
6,2,31580,7
7,2,37086,8
8,2,1604,9
9,2,22535,8


In [5]:
# Find every user's average rating for each genre and create new .csv file

# Iterate through rows of ratings
current_user_id = 0
current_user_genre_ratings = pd.DataFrame(columns=["genre", "ratings_sum", "num_of_ratings"])

for i, row in user_ratings_files_df.iterrows():

    # Next user found
    if row['user_id'] != current_user_id:
        # Calculate averages
        dictionary_row = {'user_id':current_user_id}
        for j, genre_row in current_user_genre_ratings.iterrows():
            average_rating = genre_row['ratings_sum']/genre_row['num_of_ratings']
            dictionary_row[genre_row['genre']] = average_rating

        # Save average ratings data
        average_ratings_df = pd.concat([average_ratings_df, pd.DataFrame.from_records([dictionary_row])])

        # Clear DataFrame to make available for next user and set new current user id
        current_user_genre_ratings = current_user_genre_ratings[0:0]
        current_user_id = row['user_id']

    # Find genres string for given anime_id and split into separate genre values
    current_anime_id = row['anime_id']
    current_anime_genres = anime_list_df[anime_list_df['MAL_ID'] == current_anime_id]['Genres']
    current_anime_genres_list = list(current_anime_genres.str.split(", "))

    # Fill DataFrame by iterating through genres list
    for genre in current_anime_genres_list:
        for genre_val in genre:
            # Create new genre row with genre name and other values initially set to 0 if genre does not already exist
            if not current_user_genre_ratings['genre'].isin([genre_val]).any():
                new_row_dict = {'genre':genre_val, 'ratings_sum':0, 'num_of_ratings':0}
                current_user_genre_ratings = pd.concat([current_user_genre_ratings, pd.DataFrame.from_records([new_row_dict])])

            # Add to sum of ratings and increment number of ratings
            current_user_genre_ratings.loc[current_user_genre_ratings['genre'] == genre_val, 'ratings_sum'] += row['rating']
            current_user_genre_ratings.loc[current_user_genre_ratings['genre'] == genre_val, 'num_of_ratings'] += 1

# Save data for last user
# Calculate averages
dictionary_row = {'user_id':current_user_id}
for i, genre_row in current_user_genre_ratings.iterrows():
    average_rating = genre_row['ratings_sum']/genre_row['num_of_ratings']
    dictionary_row[genre_row['genre']] = average_rating

# Save average ratings data
average_ratings_df = pd.concat([average_ratings_df, pd.DataFrame.from_records([dictionary_row])])

In [6]:
# Export to .csv file
average_ratings_df.to_csv('../../../data/average_ratings/average_ratings.csv', index=False)